In [1]:
# NSE scanner and pickler

# STATUS: Completed
# Run-time: 5 hours

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=10) # kavi tws live
# ib = IB().connect('127.0.0.1', 4002, clientId=10) # kavi IBG live

In [43]:
%%time
import numpy as np
import pandas as pd
from itertools import product
import datetime
import pickle
from math import sqrt, exp, log, erf

import os
# sd multiple for band
sigma = 2       # 2 sigma is about 95% probability
penalty = 1.1   # e.g. 1.1 is 20% above

# market
exchange = 'NSE'
tradedays = 252   # no of trading days in a year
commission = 150

#... prepare lot dataframe for underlying
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

#... Get risk-free rate from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
rate = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100


# spearate indexes and equities, eliminate discards from df_paisa
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_paisa.ibSymbol if s not in indexes+discards])

symbols = indexes + equities

#... Black-Scholes
# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undPrice, strike, dte, rate, volatality, divrate):
    ''' Gets Black Scholes output
    Args:
        (undPrice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (dte)      : Days to expiration in float
        (rate)     : dte until expiry in days
        (volatality)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
    sigTsquared = sqrt(dte/365)*volatality
    edivT = exp((-divrate*dte)/365)
    ert = exp((-rate*dte)/365)
    d1 = (log(undPrice*edivT/strike)+(rate+.5*(volatality**2))*dte/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undPrice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undPrice*edivT*iNd1, 2)
    delta = Nd1

    return {'bsmCall': callPrice, 'bsmPut': putPrice, 'bsmDelta': delta}

#***   Error catching for list comprehension ***
#_______________________________________________
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher
    Args: 
        (func) as the function
         (handle) as the lambda of function
         <*args | *kwargs> as arguments to the functions
    Outputs:
        output of the function | <np.nan> on error
    Usage:
        eggs = [1,3,0,3,2]
        [catch(lambda: 1/egg) for egg in eggs]'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan

def df_pkl(symbol):
    '''pickles df for symbols. Logic based on 1Scrip_Program
    Arg: (symbol) as string
    Returns: None'''
    
    #... Appropriate Index or Equity contract
    if symbol in equities:
        ss = Stock(symbol, exchange)
    if symbol in indexes:
        ss = Index(symbol, exchange)
        
    contract = ib.qualifyContracts(ss)[0]

    #... Get volatility, hi52 and lo52
    duration = '12 M'
    size = '1 day'
    bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                         durationStr=duration, barSizeSetting=size, 
                         whatToShow='TRADES', useRTH=True, 
                         formatDate=1, keepUpToDate=True)

    stDev = np.std(a=[b.close for b in bars], ddof=0)

    hi = max([b.high for b in bars])
    lo = min([b.low for b in bars])

    avg = np.mean([b.close for b in bars])

    #... Get the lot, margin, undPrice and dividend rate
    lot = df_paisa.loc[df_paisa.ibSymbol == symbol, 'Mlot'].item()
    margin = df_paisa.loc[df_paisa.ibSymbol == symbol, 'TotMgnPerLt'].item()

    undTicker = ib.reqMktData(contract, '456', snapshot=False, regulatorySnapshot=False)
    ib.sleep(2)
    undPrice = undTicker.close
    ib.cancelMktData(contract)

    try:
        divrate = undTicker.dividends.past12Months/undPrice
    except TypeError:
        divrate = 0.0

    #... Pickle the underlying dictionary
    dictund = {'symbol':symbol, 'bars':bars, 'stDev': stDev, 
     'hi': hi, 'lo': lo, 'avg': avg, 'lot':lot, 'margin':margin, 
     'undPrice':undPrice, 'divrate':divrate, 'rate': rate, 'contract':contract}

    with open('./zdata/_'+symbol+'.pkl', 'wb') as handle:
        pickle.dump(dictund, handle)

    #****       Option data      ****
    #________________________________

    #... Get the option chain tickers, qualify them and get the prices
    chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                          futFopExchange='', 
                          underlyingConId=contract.conId, underlyingSecType=contract.secType)

    expiries = set(*[c.expirations for c in chains])

    cds = [ib.reqContractDetails(Option(symbol, e, exchange='NSE')) for e in expiries]

    opts = [c.contract for cs in cds for c in cs]

    #... Weed out unwanted SDs and make the option chain dataframe
    # keep only the Ps and Cs outside the sigma band (95% probability)
    options = [t for t in opts 
    if ((t.strike < undPrice - stDev*sigma) & (t.right == 'P')) | 
    ((t.strike > undPrice + stDev*sigma) & (t.right == 'C'))]


    qo = [q for i in range(0, len(options), 40) for q in ib.qualifyContracts(*options[i:i+40])]

    df1 = util.df(qo)
    df1['opt_contract'] = qo

    tickers = [t for i in range(0, len(qo), 100) for t in ib.reqTickers(*qo[i:i + 100])]    
    ib.sleep(2)   # gives some time to fill the tickers    
    tickers = tickers

    df1['opt_ticker'] = tickers

    df1['dte'] = (df1.lastTradeDateOrContractMonth.apply(util.parseIBDatetime) - datetime.datetime.now().date()).dt.days

    # replace dte less than 0 with 0
    df1.loc[df1.dte<0, 'dte'] = 0

    df1['undPrice'] = undPrice
    df1['rate'] = rate
    df1['stDev'] = stDev
    df1['divrate'] = divrate

    # compute annualized volatility from running standard deviation for Black-Scholes
    df1['hv'] = [np.std(a=[b.close-b.open for b in bars[-dte:]], ddof=0)/undPrice*sqrt(tradedays) for dte in df1.dte]

    df2 = df1[['undPrice', 'strike', 'dte', 'rate', 'hv', 'divrate',
               'symbol', 'right', 'opt_contract', 'opt_ticker']]

    bsm = [catch(lambda: get_bsm(u, k, e, r, v, d)) for u, k, e, r, v, d in zip(df2.undPrice, df2.strike, df2.dte, df2.rate, df2.hv, df2.divrate)]

    # remove the Nones in the dictionary
    bsm = [{'bsmCall': np.nan, 'bsmPut': np.nan, 'bsmDelta': np.nan} if b is None else b for b in bsm]
    
    df_bsm = pd.DataFrame(list(bsm))

    df3 = pd.concat([df2, df_bsm],axis=1)

    df3['bsmPrice'] = np.where(df3.right == 'P', df3.bsmPut, df3.bsmCall)

    # Sort dataframe in ascending dte and descending strike
    df3 = df3.sort_values(by=['dte', 'strike'], ascending=[True, False]).reset_index(drop=True)

    df3['close'] = [catch(lambda: t.close) for t in df3.opt_ticker]
    df3['bid'] =  [catch(lambda: t.bid) for t in df3.opt_ticker]
    df3['ask'] = [catch(lambda: t.ask) for t in df3.opt_ticker]

    df3['lot'] = lot
    df3['margin'] = margin

    ibgreeks = [catch(lambda: (t.modelGreeks.impliedVol, t.modelGreeks.optPrice, t.modelGreeks.delta)) for t in df3.opt_ticker]
    ibgreeks = [(np.nan, np.nan, np.nan) if b is None else b for b in ibgreeks]  # for catch errors with None, replace with np.nan
    df_ib = pd.DataFrame(ibgreeks, columns=['ibiv', 'ibprice', 'ibdelta'])

    df3 = pd.concat([df3, df_ib], axis=1)

    df3['askbidavg'] = (df3.ask-df3.bid)/2

    df3['expPrice'] = round((df3[['askbidavg', 'close', 'bsmPrice']].max(axis=1)+(commission/lot))*penalty * 2, 1)/2

    df3['rom'] = (df3.expPrice*df3.lot)/df3.margin*tradedays/df3.dte
    df3['pop'] = np.where(df3.right == 'C', 1-df3.bsmDelta, df3.bsmDelta)

    cols = ['symbol', 'strike',  'dte',  'right', 'undPrice', 'pop', 'rom', 'expPrice', 'hv', 'bsmDelta',  'bsmPrice', 
            'ibiv', 'ibdelta', 'ibprice', 'close', 'bid', 'ask', 'lot', 'margin', 'opt_ticker']
    df3[cols].to_pickle('./zdata/'+symbol+'.pkl')
      
    return None

Wall time: 6.11 s


In [44]:
%%time
# Program to start from where connection failed.
fspath = './zdata/'
fs = os.listdir(fspath)

# Take only pickle files. Remove directories and files starting with underscore (for underlyings)
fs = [f for f in fs if (f[-3:] == 'pkl') & (f[0] != '_')]

# Get modified time, fail time and identify where the scrip has failed
fsmod = {f: os.path.getmtime(fspath + '/' + f) for f in fs}

failtime = max([v for k, v in fsmod.items()])
failscrip = [k[:-4] for k, v in fsmod.items() if v == failtime][0]
restartfrom = symbols.index(failscrip)

# Get the remaining pickles
s = symbols[restartfrom:]
[df_pkl(t) for t in s]
ib.disconnect()

Started to throttle requests
Stopped to throttle requests


> <timed exec>(244)df_pkl()

ipdb> l

ipdb> n
--Return--
None
> <timed exec>(244)df_pkl()

ipdb> p symbol
'ASIANPAIN'
ipdb> n
> <timed exec>(17)<listcomp>()

ipdb> p contract
*** NameError: name 'contract' is not defined
ipdb> p duration
*** NameError: name 'duration' is not defined
ipdb> w
  c:\users\kashir\appdata\local\programs\python\python37-32\lib\runpy.py(193)_run_module_as_main()
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
    194 

  c:\users\kashir\appdata\local\programs\python\python37-32\lib\runpy.py(85)_run_code()
     84                        __spec__ = mod_spec)
---> 85     exec(code, run_globals)
     86     return run_globals

  c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py(16)<module>()
     14 
     15     from ipykernel import kernelapp as app
---> 16     app.launch_new_instance()

  c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-pac

ipdb> n


Started to throttle requests
Stopped to throttle requests


> <timed exec>(244)df_pkl()

ipdb> symbol
'AUROPHARM'
ipdb> n
--Return--
None
> <timed exec>(244)df_pkl()

ipdb> symbol
'AUROPHARM'
ipdb> n
> <timed exec>(17)<listcomp>()

ipdb> symbol
*** NameError: name 'symbol' is not defined
ipdb> n


Started to throttle requests
Stopped to throttle requests


> <timed exec>(244)df_pkl()

ipdb> symbol
'AXISBANK'
ipdb> n
--Return--
None
> <timed exec>(244)df_pkl()

ipdb> symbol
'AXISBANK'
ipdb> n
> <timed exec>(17)<listcomp>()

ipdb> symbol
*** NameError: name 'symbol' is not defined
ipdb> n
TypeError: 'NoneType' object does not support item assignment
> <timed exec>(17)<listcomp>()

ipdb> symbol
*** NameError: name 'symbol' is not defined
ipdb> u
None
> <timed exec>(17)<module>()

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-packages\ipython\core\magics\execution.py(1271)time()
   1270             try:
-> 1271                 exec(code, glob, local_ns)
   1272             except:

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-packages\ipython\core\magic.py(187)<lambda>()
    186     def magic_deco(arg):
--> 187         call = lambda f, *a, **k: f(*a, **k)
    188 

ipdb> u
> <decorator-gen-63>(2)time()

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel\kernelbase.py(534)execute_request()
    533                 code, silent, store_history,
--> 534                 user_expressions, allow_stdin,
    535             )

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-packages\tornado\gen.py(326)wrapper()
    325                     orig_stack_contexts = stack_context._state.contexts
--> 326                     yielded = next(result)
    327                     if stack_context._state.contexts is not orig_stack_contexts:

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel\kernelbase.py(267)dispatch_shell()
    266             try:
--> 267                 yield gen.maybe_future(handler(stream, idents, msg))
    268             except Exception:

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-packages\tornado\gen.py(326)wrapper()
    325       

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py(16)<module>()
     14 
     15     from ipykernel import kernelapp as app
---> 16     app.launch_new_instance()

ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\runpy.py(85)_run_code()
     84                        __spec__ = mod_spec)
---> 85     exec(code, run_globals)
     86     return run_globals

ipdb> p symbol
*** NameError: name 'symbol' is not defined
ipdb> u
> c:\users\kashir\appdata\local\programs\python\python37-32\lib\runpy.py(193)_run_module_as_main()
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
    194 

ipdb> u
*** Oldest frame
ipdb> u
*** Oldest frame
ipdb> u
*** Oldest frame
ipdb> u
*** Oldest frame
ipdb> n


TypeError: 'NoneType' object does not support item assignment

In [ ]:
symbols = symbols[restartfrom:]
[df_pkl(t) for t in symbols]

In [ ]:
%%time
# New pickles of symbols in blocks of 50
[df_pkl(t) for i in range(0, len(symbols), 50) for t in symbols[i: i+50]]
ib.disconnect()